In [1]:
import logging

In [2]:
logging.basicConfig(level="INFO", format="[%(name)s - %(levelname)s] %(message)s")

In [3]:
ROOT = logging.getLogger()

In [4]:
import pandas as pd

In [5]:
import sanger_sequencing

In [6]:
from pandas import read_excel

def tube_samples(filepath):
    """Read the particular excel file into a pandas.DataFrame."""
    df = read_excel(filepath, converters={
        "Primer ID": str,
        "Plasmid ID": str
    })
    df.dropna(how="any", inplace=True)
    return df

In [7]:
old_template = tube_samples("../sanger-service/cfb/tests/data/Mix2Seq_SA00360224_sample.xls")
old_template.head()

,Rack Position,Tube Code,Your Name,Plasmid ID,Primer ID
0,A1,FR14453230,cpgfp_a_14617,7138,14617
1,B1,FR14453231,cpgfp_a_20691,7138,20691
2,C1,FR14453232,cpgfp_a_20692,7138,20692
3,D1,FR14453233,cpgfp_a_14619,7138,14619
4,E1,FR14453234,cpgfp_b_14617,7138,14617


In [8]:
from glob import glob

In [9]:
from Bio import SeqIO

In [10]:
from os.path import splitext, basename

In [11]:
def genbank_db(samples):
    db = dict()
    for plasmid_id in samples["Plasmid ID"].unique():
        path = glob(f"../sanger-service/cfb/tests/data/pCfB{plasmid_id}*.gbk")[0]
        db[plasmid_id] = splitext(basename(path))[0], SeqIO.read(path, "gb")
    return db

In [12]:
plasmids = genbank_db(old_template)

In [13]:
plasmids

{'7138': ('pCfB7138 (pIntC_3_PrTEFint-CpFAH-hrGFPTLip2)',
  SeqRecord(seq=Seq('GAATGCGTGCGATAAAAAACTGTAGTAGTGTGGTGATGGAGTCATAACCCGCTC...CGC', IUPACAmbiguousDNA()), id='pCfB4366\\(pIntG-Nat-TPex20-TLip2).0', name='pCfB7138_(pIntC_3_PrTEFint-CpFAH-hrGFPTLip2)', description='', dbxrefs=[])),
 '7139': ('pCfB7139 (pIntC_3_PrTEFint-SmOhy-hrGFP-TLip2)',
  SeqRecord(seq=Seq('GAATGCGTGCGATAAAAAACTGTAGTAGTGTGGTGATGGAGTCATAACCCGCTC...CGC', IUPACAmbiguousDNA()), id='pCfB4366\\(pIntG-Nat-TPex20-TLip2).0', name='pCfB7139_(pIntC_3_PrTEFint-SmOhy-hrGFP-TLip2)', description='', dbxrefs=[])),
 '7140': ('pCfB7140 (pIntC_3_PrTEFint-LaLhy-hrGFP-TLip2)',
  SeqRecord(seq=Seq('GAATGCGTGCGATAAAAAACTGTAGTAGTGTGGTGATGGAGTCATAACCCGCTC...CGC', IUPACAmbiguousDNA()), id='pCfB4366\\(pIntG-Nat-TPex20-TLip2).0', name='pCfB7140_(pIntC_3_PrTEFint-LaLhy-hrGFP-TLip2)', description='', dbxrefs=[])),
 '7135': ('pCfB7135 (pIntD_1-_PrTEFint-CpFAH-PTS1-TLip2)',
  SeqRecord(seq=Seq('GAATGCGTGCGATAAAAAACTGTAGTAGTGTGGTGATGGAGTCATA

In [14]:
from os.path import basename, splitext

def ab1_filter(filepath) -> bool:
    """Select non-hidden ab1 files."""
    if basename(filepath).startswith('.'):
        return False
    return splitext(filepath)[1].lower().endswith('.ab1')

In [15]:
def get_tube_code(filepath) -> str:
    """Extract the tube code part from a filename."""
    return splitext(basename(filepath))[0].split("_")[-1]

In [16]:
from zipfile import ZipFile
from io import BytesIO

def parse_tube_ab1(archive: ZipFile,
                   tube_codes):
    """Read particular ab1 sequences from an archive."""
    sequences = dict()
    # Create a tube code -> file name map from the archive's contents.
    names = {get_tube_code(f): f for f in filter(
        ab1_filter, archive.namelist())}
    # Try to extract all desired sequences.
    for code in tube_codes:
        if code not in names:
            LOGGER.error("No ab1 file found for tube with code '%s'.", code)
            continue
        with archive.open(names[code]) as file_handle:
            # We use `SeqIO.read` because ab1 files contain only a single
            # sequence record.
            sequences[code] = SeqIO.read(BytesIO(file_handle.read()), 'abi')
    return sequences

In [17]:
def extract_sequences(template, archive):
    return parse_tube_ab1(archive, template['Tube Code'].unique())

In [18]:
zip_path = "../sanger-service/cfb/tests/data/11104089228-1_SCF_SEQ_ABI.zip"

In [19]:
with ZipFile(zip_path) as archive:
    samples = extract_sequences(old_template, archive)

In [20]:
from pandas import DataFrame

In [21]:
template = DataFrame({
    "plasmid": old_template["Plasmid ID"],
    "primer": old_template["Primer ID"],
    "sample": old_template["Tube Code"]
})

In [22]:
new_plasmids = {pid: seq for pid, (name, seq) in plasmids.items()}

In [23]:
template.head()

,plasmid,primer,sample
0,7138,14617,FR14453230
1,7138,20691,FR14453231
2,7138,20692,FR14453232
3,7138,14619,FR14453233
4,7138,14617,FR14453234


In [24]:
from sanger_sequencing.api import plasmid_report

In [25]:
ROOT.setLevel(logging.INFO)

In [26]:
report = plasmid_report("7138", new_plasmids["7138"], template[:5], samples)

[sanger_sequencing.api - INFO] Analyze plasmid '7138'.
[sanger_sequencing.api - INFO] Analyze sample 'FR14453230'.
[sanger_sequencing.api - INFO] Analyze sample 'FR14453231'.
[sanger_sequencing.api - INFO] Analyze sample 'FR14453232'.
[sanger_sequencing.api - INFO] Analyze sample 'FR14453233'.
[sanger_sequencing.analysis.alignment - INFO] Trying reverse complement!
[sanger_sequencing.api - INFO] Analyze sample 'FR14453234'.
[sanger_sequencing.analysis.summary - INFO] Assessing 5 conflicts.
[sanger_sequencing.analysis.summary - INFO] Assessing 5 conflicts.
[sanger_sequencing.analysis.summary - INFO] Assessing 5 conflicts.
[sanger_sequencing.analysis.summary - INFO] Assessing 5 conflicts.
[sanger_sequencing.analysis.summary - INFO] Assessing 5 conflicts.


In [27]:
report["samples"][0]["conflicts"]

[]

In [28]:
for r in report["samples"]:
    print(r["conflicts"])

[]
[]
[OrderedDict([('plasmid_pos', 2569.0), ('sample_pos', nan), ('plasmid_chr', 'C'), ('sample_chr', '-'), ('quality', nan), ('sample', 'FR14453232'), ('primer', '20692'), ('type', 'deletion'), ('avgQuality', 'high'), ('confirmed', 0), ('invalidated', 2), ('featuresHit', [('promoter', ['PrTEFintron'])]), ('effect', [])]), OrderedDict([('plasmid_pos', nan), ('sample_pos', 713.0), ('plasmid_chr', '-'), ('sample_chr', 'T'), ('quality', 13.0), ('sample', 'FR14453232'), ('primer', '20692'), ('type', 'insertion'), ('avgQuality', 'high'), ('confirmed', 0), ('invalidated', 3), ('featuresHit', [('CDS', ['CpFAH12_codoptYL'])]), ('effect', ['Frame shift'])]), OrderedDict([('plasmid_pos', 2981.0), ('sample_pos', nan), ('plasmid_chr', 'G'), ('sample_chr', '-'), ('quality', nan), ('sample', 'FR14453232'), ('primer', '20692'), ('type', 'deletion'), ('avgQuality', 'high'), ('confirmed', 0), ('invalidated', 3), ('featuresHit', [('CDS', ['CpFAH12_codoptYL'])]), ('effect', ['Frame shift'])]), OrderedDi